In [152]:
import pandas as pd
import numpy as np
import pymysql

In [153]:
#从数据库中获取数据

#连接数据库
db = pymysql.connect(
    host='gz-cdb-9avl6ee9.sql.tencentcdb.com',
    port=57122,
    user='read',
    passwd='feier222X',
    charset='utf8'
)

#创建游标
cursor = db.cursor()

#查看数据库
cursor.execute("show databases")

#接收查询结果
DBs = cursor.fetchall() #返回的数据结果是元组
print("databases are:",DBs)


#进入water数据库
cursor.execute("use water")
#查看数据表
cursor.execute("show tables")

#查看数据表
water_tables = cursor.fetchall()
# print("tables are:",water_tables)


part_table_name = 'water_data_'

table_name = part_table_name + IMEI_number


IMEI_number = 'F1B40B60252E'
start_time = 1658705400
end_time = 1658791800
#to do prams
#查询语句


sql = 'select * from ' + table_name + ' where time >= %s and time < %s' 
# print("sql ----",sql)
args = start_time,end_time

# sql = "select * from water_data_FA78EFAFF9D0 where time >= 1661868121 and time <1661954521"


#存储文件名和路径
# path_csv_filename = '/Users/qinjianquan/Desktop/tmpfile/water_data_FA78EFAFF9D0.csv'
path_temp01_filename = '/Users/qinjianquan/Desktop/FEIER/GCYY/raw_data.csv'
# path_excel_filename = '/Users/qinjianquan/Desktop/tmpfile/water_data_FA78EFAFF9D0.xlsx'


#执行查询并返回结果
cursor.execute(sql,args)
#返回查询到的数据
rows = cursor.fetchall()

#转换数据对象
data_table = pd.DataFrame(list(rows),columns = ['时间', 'ut', 'dn', 'i', '心率', '低压', '高压', '前面积', '后面积', 'RR', 'step', 'acc_x', 'acc_z', 'acc_y'] )
display("raw_data_table:",data_table)

#重命名
df_csv =data_table

#整理数据，计算体动
df_csv["add_xyz"] = df_csv["acc_x"] + df_csv["acc_z"] + df_csv["acc_y"]

df_csv["sub_xyz"] = df_csv["add_xyz"] - df_csv["add_xyz"].shift(-1)

df_csv["dsub_xyz"] = df_csv["sub_xyz"] - df_csv["sub_xyz"].shift(-1)

df_csv["dsub_xyz/10"] = df_csv["dsub_xyz"].div(10)
df_csv["round"] = df_csv["dsub_xyz/10"].round()
df_csv["abs"] = df_csv["round"].abs()
df_csv["abs_52"] = df_csv["abs"]

df_csv.loc[df_csv["abs"] > 27,'abs_52'] = df_csv["abs_52"].sub(27)
df_csv["体动"] = df_csv["abs_52"]

df_csv.loc[df_csv["体动"] > 10000,'体动'] = df_csv["体动"].div(10)
df_csv.loc[df_csv["体动"] > 10000,'体动'] = df_csv["体动"].div(10)

display("computed file:",df_csv)

#数据类型转换
df_csv['时间']=pd.to_datetime(df_csv['时间'],unit='s',origin=pd.Timestamp('1970-01-01'))

#删除无效行
df_csv.drop(df_csv.tail(2).index,inplace=True)

#删除多余列
df_csv=df_csv.drop(["abs","abs_52","round","ut","dn","i","step","acc_x","acc_y","acc_z","add_xyz","sub_xyz","dsub_xyz","dsub_xyz/10"],axis=1)

#保存文件
df_csv.to_csv(path_temp01_filename,index=False,header=True) 

# display("result csv file:",df_csv)
df = df_csv


databases are: (('information_schema',), ('water',))


'raw_data_table:'

,时间,ut,dn,i,心率,低压,高压,前面积,后面积,RR,step,acc_x,acc_z,acc_y
0,1658705400,1658743662,80,19,92,69,112,408,1351,672,598,-1070,7633,1349
1,1658705401,1658743662,80,19,92,69,112,346,1141,644,598,-1198,7546,1460
2,1658705402,1658743662,80,19,92,69,112,455,1548,644,598,-1260,7462,1517
3,1658705403,1658743662,80,19,92,69,111,483,1548,652,598,-1126,7613,1462
4,1658705404,1658743662,80,19,92,69,111,471,1531,684,598,-992,7494,1494
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78620,1658790775,1658792586,31,580,0,0,0,0,0,0,1738,1311,-779,8622
78621,1658790776,1658792586,31,580,0,0,0,0,0,0,1738,1855,-1124,7533
78622,1658790777,1658792586,31,580,0,0,0,0,0,0,1738,4,-957,8084
78623,1658790778,1658792586,31,580,0,0,0,0,0,0,1738,-7217,-3696,-1302


'computed file:'

,时间,ut,dn,i,心率,低压,高压,前面积,后面积,RR,...,acc_z,acc_y,add_xyz,sub_xyz,dsub_xyz,dsub_xyz/10,round,abs,abs_52,体动
0,1658705400,1658743662,80,19,92,69,112,408,1351,672,...,7633,1349,7912,104.0,15.0,1.5,2.0,2.0,2.0,2.0
1,1658705401,1658743662,80,19,92,69,112,346,1141,644,...,7546,1460,7808,89.0,319.0,31.9,32.0,32.0,32.0,32.0
2,1658705402,1658743662,80,19,92,69,112,455,1548,644,...,7462,1517,7719,-230.0,-183.0,-18.3,-18.0,18.0,18.0,18.0
3,1658705403,1658743662,80,19,92,69,111,483,1548,652,...,7613,1462,7949,-47.0,-91.0,-9.1,-9.0,9.0,9.0,9.0
4,1658705404,1658743662,80,19,92,69,111,471,1531,684,...,7494,1494,7996,44.0,-80.0,-8.0,-8.0,8.0,8.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78620,1658790775,1658792586,31,580,0,0,0,0,0,0,...,-779,8622,9154,890.0,-243.0,-24.3,-24.0,24.0,24.0,24.0
78621,1658790776,1658792586,31,580,0,0,0,0,0,0,...,-1124,7533,8264,1133.0,-18213.0,-1821.3,-1821.0,1821.0,1769.0,1769.0
78622,1658790777,1658792586,31,580,0,0,0,0,0,0,...,-957,8084,7131,19346.0,17900.0,1790.0,1790.0,1790.0,1738.0,1738.0
78623,1658790778,1658792586,31,580,0,0,0,0,0,0,...,-3696,-1302,-12215,1446.0,NaN,NaN,NaN,NaN,NaN,NaN


In [154]:
# pd_reader = pd.read_csv("./raw-61data.csv")
# file = 'tmp01'
file = 'raw_data'

pd_reader = pd.read_csv(file+'.csv')


# print(pd_reader)

# # df = pd_reader.drop(['时间'], axis=1)
df = pd_reader.copy()
# print("df:",df)
display(df)

# 调整1，去除体动大于100的数据
df = df.drop(df[(df['体动'] > 100)].index)
display(df)

# temp data cleaning
# df = df.drop(df[(df['心率'] == 0) | (df['低压'] == 0) | (df['高压'] == 0)].index)
# data clean, replace 0 by  1
df.replace(to_replace = 0, value = 1, inplace=True)
df = df.drop(columns=['X', 'Y', 'Z', '前面积','后面积','体动'], errors='ignore')
df = df.drop(df[(df['心率'] == 1) | (df['低压'] == 1) | (df['高压'] == 1) | (df['RR'] == 1)].index)

df = df.drop(df[(df['低压'] < 40) | (df['高压'] < 80)].index)
# witoutCol = '低压'
# df = df.drop(columns=[witoutCol])
df.reset_index(drop=True, inplace=True)
display(df)

,时间,心率,低压,高压,前面积,后面积,RR,体动
0,2022-07-24 23:30:00,92,69,112,408,1351,672,2.0
1,2022-07-24 23:30:01,92,69,112,346,1141,644,32.0
2,2022-07-24 23:30:02,92,69,112,455,1548,644,18.0
3,2022-07-24 23:30:03,92,69,111,483,1548,652,9.0
4,2022-07-24 23:30:04,92,69,111,471,1531,684,8.0
...,...,...,...,...,...,...,...,...
78618,2022-07-25 23:12:53,0,0,0,0,0,0,64.0
78619,2022-07-25 23:12:54,0,0,0,0,0,0,50.0
78620,2022-07-25 23:12:55,0,0,0,0,0,0,24.0
78621,2022-07-25 23:12:56,0,0,0,0,0,0,1769.0


,时间,心率,低压,高压,前面积,后面积,RR,体动
0,2022-07-24 23:30:00,92,69,112,408,1351,672,2.0
1,2022-07-24 23:30:01,92,69,112,346,1141,644,32.0
2,2022-07-24 23:30:02,92,69,112,455,1548,644,18.0
3,2022-07-24 23:30:03,92,69,111,483,1548,652,9.0
4,2022-07-24 23:30:04,92,69,111,471,1531,684,8.0
...,...,...,...,...,...,...,...,...
78615,2022-07-25 23:12:50,0,0,0,0,0,0,62.0
78616,2022-07-25 23:12:51,0,0,0,0,0,0,40.0
78618,2022-07-25 23:12:53,0,0,0,0,0,0,64.0
78619,2022-07-25 23:12:54,0,0,0,0,0,0,50.0


,时间,心率,低压,高压,RR
0,2022-07-24 23:30:00,92,69,112,672
1,2022-07-24 23:30:01,92,69,112,644
2,2022-07-24 23:30:02,92,69,112,644
3,2022-07-24 23:30:03,92,69,111,652
4,2022-07-24 23:30:04,92,69,111,684
...,...,...,...,...,...
57960,2022-07-25 22:53:42,95,73,128,244
57961,2022-07-25 22:54:11,95,74,130,950
57962,2022-07-25 22:54:15,95,74,130,950
57963,2022-07-25 22:54:16,95,74,130,947


## 一 计算健康标尺

In [155]:
# Geometric Mean of the column in dataframe
from scipy import stats
import scipy
print(len(df.columns)) 
##scipy.stats.gmean(df.iloc[:,1:7],axis=0)
# 结构时中
GEOMEAN1 = scipy.stats.gmean(df.iloc[:,1:len(df.columns)],axis=1)
print(GEOMEAN1)
import math
# M个数据组结构时中中最大值与最小值的定量差异（最大值与最小值比值以0.618为底取对数，在取绝对值）
QDMM1 = abs(math.log(max(GEOMEAN1)/min(GEOMEAN1),0.618))
print(QDMM1)

5
[147.84479977 146.28008433 146.28008433 ... 171.65464921 171.51897161
 171.56423325]
2.444425882916705


In [156]:
def calculateQDMMByFixedYinYang(df_yang, df_yin):
	QDMM_MAX = QDMM1
	colname_max = ''
	index_max = 0
	# display("yin", df_yin)
	if len(df_yin) > 0:
		df_temp = pd.concat([df_yang, df_yin], axis=1, join='inner')		
		for (index, colname) in enumerate(df_yin):
			print('df_yang: ', index, colname)
			if index !=0:
				df_yin[colname] = 1/df_yin[colname]
				GEOMEAN_temp = scipy.stats.gmean(df_temp.iloc[:,1:len(df_temp.columns)],axis=1)
				# print(GEOMEAN_temp)
				# M个数据组结构时中中最大值与最小值的定量差异（最大值与最小值比值以0.618为底取对数，在取绝对值）
				QDMM_temp = abs(math.log(max(GEOMEAN_temp)/min(GEOMEAN_temp),0.618))
				# print(QDMM_temp, index, colname)
				if QDMM_temp > QDMM_MAX:
					QDMM_MAX = QDMM_temp
					colname_max = colname
					index_max = index	
					df_drop = df_temp[colname]

In [157]:
def calculateQDMM(df_yang, df_yin):
	QDMM_MAX = QDMM1
	colname_max = ''
	index_max = 0
	# display("yin", df_yin)
	if len(df_yin) > 0:
		df_temp = pd.concat([df_yang, df_yin], axis=1, join='inner')		
		for (index, colname) in enumerate(df_yang):
			print('df_yang: ', index, colname)
			if index !=0:
				df_temp[colname] = 1/df_temp[colname]
				GEOMEAN_temp = scipy.stats.gmean(df_temp.iloc[:,1:len(df_temp.columns)],axis=1)
				# print(GEOMEAN_temp)
				# M个数据组结构时中中最大值与最小值的定量差异（最大值与最小值比值以0.618为底取对数，在取绝对值）
				QDMM_temp = abs(math.log(max(GEOMEAN_temp)/min(GEOMEAN_temp),0.618))
				# print(QDMM_temp, index, colname)
				if QDMM_temp > QDMM_MAX:
					QDMM_MAX = QDMM_temp
					colname_max = colname
					index_max = index	
					df_drop = df_temp[colname]
	else:
		df_temp = df_yang.copy()
		for (index, colname) in enumerate(df_temp):
			print('no df_yin: ', index, colname)
			if index !=0:
				df_temp[colname] = 1/df_temp[colname]
				GEOMEAN_temp = scipy.stats.gmean(df_temp.iloc[:,1:len(df_temp.columns)],axis=1)
				# print(GEOMEAN_temp)
				# M个数据组结构时中中最大值与最小值的定量差异（最大值与最小值比值以0.618为底取对数，在取绝对值）
				QDMM_temp = abs(math.log(max(GEOMEAN_temp)/min(GEOMEAN_temp),0.618))
				# print(QDMM_temp, index, colname)
				if QDMM_temp > QDMM_MAX:
					QDMM_MAX = QDMM_temp
					colname_max = colname
					index_max = index	
					df_drop = df_temp[colname]
	# print(QDMM_temp)
	if (index_max != 0):
		print(QDMM_MAX, colname_max, index_max)
		df_yang = df_yang.drop([colname_max], axis=1)
		# display(df_yang)
		if len(df_yin) > 0:
			df_yin  = pd.concat([df_yin, df_drop], axis=1, join='inner')
		else:
			df_yin = df_drop
		# display(df_yin)
		# calculateQDMM(df_yang, df_yin)
	# else:
	# 	print(QDMM_MAX, colname_max, index_max)
	# 	df_temp = pd.concat([df_yang, df_yin], axis=1, join='inner')
	# 	display(df_yang)
	return df_yang, df_yin, QDMM_temp, index_max	

## 二 排除异常数据组

In [158]:
def dataClean(df_yang, df_yin):
	df_temp = pd.concat([df_yang, df_yin], axis=1, join='inner')
	display(df_temp)
	# df_temp = df.copy()
	for (index, colname) in enumerate(df_temp):
		if index !=0:
			# 1.在GCYY数据流中计算每个参数的功能时中
			GEOMEAN_func_temp = scipy.stats.gmean(df_temp.iloc[:,index],axis=0)
			# print(GEOMEAN_func_temp)
			# 2.每个参数纵向除以自己的功能时中
			df_temp[colname] = df_temp[colname]/GEOMEAN_func_temp
		
	# df_temp = df_temp.append(df_temp.sum(axis=1, numeric_only=True), ignore_index=True)
	dataframe_sum = df_temp.sum(axis=1, numeric_only=True)
	# 3.每个数据组横向计算所有参数的和，称为数据和
	print(dataframe_sum)
	df_temp["sum"] = dataframe_sum
	
	# 4.计算每个参数横向除以自己的数据和，称为占比
	for (index, colname) in enumerate(df_temp):
		if index !=0 and colname != "sum":
			df_temp[colname] = df_temp[colname]/df_temp["sum"]
	# print(df)
	# 计算每个数据组横向所有占比的结构时中
	GEOMEAN2 = scipy.stats.gmean(df_temp.iloc[:,1:len(df_temp.columns)-2],axis=1)
	# print(GEOMEAN2)
	# 计算所有结构时中的几何均值GM
	GEOMEAN2_GM = scipy.stats.gmean(GEOMEAN2)
	print(GEOMEAN2_GM)
	# print(GEOMEAN2_GM)
	# 计算每个结构时中与GM的定量差异，计算相继两个结构时中的定量差异，删除其中定量差异大于0.1610的数据组
	import numpy as np
	arr = np.array(GEOMEAN2)
	# todo: #1 check
	qdmmArray = abs(np.log(arr/GEOMEAN2_GM)/np.log(0.618))
	print(qdmmArray)
	qdmmDiffArray = np.diff(qdmmArray)
	print(qdmmDiffArray)
	indexrResult = np.where(abs(qdmmDiffArray) > 0.1610)
	# indexrResult = np.where(abs(np.log(arr/GEOMEAN2_GM)/np.log(0.618)) > 0.1610)
	print(len(indexrResult[0]))
	# tt = []
	# for i in range(arr.shape[0]-1):		
	# 	# QDMM_temp = abs(math.log(arr[i+1]/arr[i],0.618))
	# 	QDMM_temp = abs(np.log(arr[i+1]/arr[i])/np.log(0.618))
	# 	if (QDMM_temp > 0.1610):
	# 		tt.append(i)
	# 		tt.append(i+1)
			# np.append(indexrResult[0], i)
			# np.append(indexrResult[0], i+1)
	# print("tt", len(tt))
	# print(len(indexrResult[0]))
	df_temp.drop(indexrResult[0], axis=0, inplace=True)
	df_yang.drop(indexrResult[0], axis=0, inplace=True)
	df_yin.drop(indexrResult[0], axis=0, inplace=True)
	# df.drop([5,6], axis=0, inplace=True)
	return (df_yang, df_yin)


In [159]:
# QDMM_MAX = QDMM1
# df_yang, df_yin, QDMM_temp, index_max = calculateQDMM(df, pd.DataFrame([]))
# display(df_yang)
# display(df_yin)
# while index_max != 0:		
# 	df_yang, df_yin, QDMM_temp, index_max = calculateQDMM(df_yang, df_yin)

# special case for specified yin and yang
for (index, colname) in enumerate(df):
	print('df: ', index, colname)
	if index !=0 and colname != 'RR':
		df[colname] = 1/df[colname]
df_yang = df.drop(['心率','低压','高压','时间'], axis=1, errors='ignore')
df_yin = df.drop(['RR','时间'], axis=1)

display(df_yang)
display(df_yin)

df_yang, df_yin = dataClean(df_yang, df_yin)


df:  0 时间
df:  1 心率
df:  2 低压
df:  3 高压
df:  4 RR


,RR
0,672
1,644
2,644
3,652
4,684
...,...
57960,244
57961,950
57962,950
57963,947


,心率,低压,高压
0,0.010870,0.014493,0.008929
1,0.010870,0.014493,0.008929
2,0.010870,0.014493,0.008929
3,0.010870,0.014493,0.009009
4,0.010870,0.014493,0.009009
...,...,...,...
57960,0.010526,0.013699,0.007812
57961,0.010526,0.013514,0.007692
57962,0.010526,0.013514,0.007692
57963,0.010526,0.013514,0.007692


,RR,心率,低压,高压
0,672,0.010870,0.014493,0.008929
1,644,0.010870,0.014493,0.008929
2,644,0.010870,0.014493,0.008929
3,652,0.010870,0.014493,0.009009
4,684,0.010870,0.014493,0.009009
...,...,...,...,...
57960,244,0.010526,0.013699,0.007812
57961,950,0.010526,0.013514,0.007692
57962,950,0.010526,0.013514,0.007692
57963,947,0.010526,0.013514,0.007692


0        675.259736
1        647.259736
2        647.259736
3        655.269876
4        687.269876
            ...    
57960    247.027386
57961    952.998887
57962    952.998887
57963    949.998887
57964    950.998887
Length: 57965, dtype: float64
0.0018711873387640824
[0.35158159 0.26358519 0.26358519 ... 1.1734537  1.1669024  1.16908846]
[-0.08799641  0.          0.02555654 ...  0.         -0.0065513
  0.00218606]
8998


# 三 计算健康标尺中的非常显著性差异点

In [160]:
# 再重复以上两个大步骤，直到结构不再变化。
# TODO: temp remove
# df_yang, df_yin, QDMM_temp, index_max = calculateQDMM(df_yang, df_yin)
# display(df_yang)
# display(df_yin)
# 在排除异常数据组之后，将GCYY数据流中的GCYY排序，计算所有GCYY的几何均值GM，
df_all = pd.concat([df_yang, df_yin], axis=1, join='inner')
print(len(df_all.columns)) 
GCYY = scipy.stats.gmean(df_all.iloc[:,1:len(df_all.columns)],axis=1)
print(GCYY)
GM = scipy.stats.gmean(GCYY)
print(GM)
# 计算每个GCYY与GM的定量差异，计算相继两个GCYY的定量差异，
# 标出其中定量差异大于0.805的数据组，其中GCYY较低的标绿色，GCYY较高的标橙色
import numpy as np
arr_GCYY = np.array(GCYY)

# qdmmArray = abs(np.log(arr_GCYY/GM)/np.log(0.618))
# print(qdmmArray)
# qdmmDiffArray = np.diff(qdmmArray)
# print(qdmmDiffArray)
# indexrResult = np.where(abs(qdmmDiffArray) > 0.805)
# print(indexrResult)
# print(arr_GCYY[indexrResult[0]])
# indexrGreen = np.where((arr_GCYY[indexrResult[0]] < GM))
# indexrOrange = np.where((arr_GCYY[indexrResult[0]] > GM))

indexrGreen = np.where((abs(np.log(arr_GCYY/GM)/np.log(0.618)) > 0.805) & (arr_GCYY < GM))
indexrOrange = np.where((abs(np.log(arr_GCYY/GM)/np.log(0.618)) > 0.805) & (arr_GCYY > GM))
print(np.where((abs(np.log(arr_GCYY/GM)/np.log(0.618)) > 0.805)))
print(len(indexrGreen[0]))
print(len(indexrOrange[0]))

4
[0.01120422 0.01120422 0.01120422 ... 0.01030466 0.01030466 0.01030466]
0.010341268779063656
(array([], dtype=int64),)
0
0


In [161]:
# write to csv
display(df)
import os  
os.makedirs('subfolder', exist_ok=True)  
df_all['GCYY'] = GCYY
# df_all['time'] = pd.to_datetime(df['时间'], format='%Y-%m-%d %H:%M:%S')
df_all['time'] = pd.to_datetime(df['时间'], format='%Y-%m-%d %H:%M:%S')
df_all = df_all.drop(df_all[(df_all['GCYY'] == 1)].index)
# df_csv = pd.concat([df_all, df_yin], axis=1, join='inner')
output = 'subfolder/' + 'data_analysis_report' + '.csv'
print(output)
df_csv = df_all.copy()
display(df_csv)
for (index, colname) in enumerate(df_csv):
	print('df_csv: ', index, colname)
	if ((colname == '心率') | (colname == '低压') | (colname == '高压')):
		df_csv[colname] = 1/df_csv[colname]
display(df_csv)

# 调整2， 计算每一分钟gcyy的平均值，只评估安静状态的gcyy
df_csv['time'] = df_csv['time'].dt.floor('T')
display(df_csv)
grouped_df = df_csv.groupby(['time'])
mean_df = grouped_df.mean()
mean_df = mean_df.reset_index()
display(mean_df)

mean_df.to_csv(output, index=False)

,时间,心率,低压,高压,RR
0,2022-07-24 23:30:00,0.010870,0.014493,0.008929,672
1,2022-07-24 23:30:01,0.010870,0.014493,0.008929,644
2,2022-07-24 23:30:02,0.010870,0.014493,0.008929,644
3,2022-07-24 23:30:03,0.010870,0.014493,0.009009,652
4,2022-07-24 23:30:04,0.010870,0.014493,0.009009,684
...,...,...,...,...,...
57960,2022-07-25 22:53:42,0.010526,0.013699,0.007812,244
57961,2022-07-25 22:54:11,0.010526,0.013514,0.007692,950
57962,2022-07-25 22:54:15,0.010526,0.013514,0.007692,950
57963,2022-07-25 22:54:16,0.010526,0.013514,0.007692,947


subfolder/data_analysis_report.csv


,RR,心率,低压,高压,GCYY,time
0,672,0.010870,0.014493,0.008929,0.011204,2022-07-24 23:30:00
1,644,0.010870,0.014493,0.008929,0.011204,2022-07-24 23:30:01
2,644,0.010870,0.014493,0.008929,0.011204,2022-07-24 23:30:02
3,652,0.010870,0.014493,0.009009,0.011238,2022-07-24 23:30:03
4,684,0.010870,0.014493,0.009009,0.011238,2022-07-24 23:30:04
...,...,...,...,...,...,...
57959,244,0.010526,0.013699,0.007812,0.010405,2022-07-25 22:53:41
57961,950,0.010526,0.013514,0.007692,0.010305,2022-07-25 22:54:11
57962,950,0.010526,0.013514,0.007692,0.010305,2022-07-25 22:54:15
57963,947,0.010526,0.013514,0.007692,0.010305,2022-07-25 22:54:16


df_csv:  0 RR
df_csv:  1 心率
df_csv:  2 低压
df_csv:  3 高压
df_csv:  4 GCYY
df_csv:  5 time


,RR,心率,低压,高压,GCYY,time
0,672,92.0,69.0,112.0,0.011204,2022-07-24 23:30:00
1,644,92.0,69.0,112.0,0.011204,2022-07-24 23:30:01
2,644,92.0,69.0,112.0,0.011204,2022-07-24 23:30:02
3,652,92.0,69.0,111.0,0.011238,2022-07-24 23:30:03
4,684,92.0,69.0,111.0,0.011238,2022-07-24 23:30:04
...,...,...,...,...,...,...
57959,244,95.0,73.0,128.0,0.010405,2022-07-25 22:53:41
57961,950,95.0,74.0,130.0,0.010305,2022-07-25 22:54:11
57962,950,95.0,74.0,130.0,0.010305,2022-07-25 22:54:15
57963,947,95.0,74.0,130.0,0.010305,2022-07-25 22:54:16


,RR,心率,低压,高压,GCYY,time
0,672,92.0,69.0,112.0,0.011204,2022-07-24 23:30:00
1,644,92.0,69.0,112.0,0.011204,2022-07-24 23:30:00
2,644,92.0,69.0,112.0,0.011204,2022-07-24 23:30:00
3,652,92.0,69.0,111.0,0.011238,2022-07-24 23:30:00
4,684,92.0,69.0,111.0,0.011238,2022-07-24 23:30:00
...,...,...,...,...,...,...
57959,244,95.0,73.0,128.0,0.010405,2022-07-25 22:53:00
57961,950,95.0,74.0,130.0,0.010305,2022-07-25 22:54:00
57962,950,95.0,74.0,130.0,0.010305,2022-07-25 22:54:00
57963,947,95.0,74.0,130.0,0.010305,2022-07-25 22:54:00


,time,RR,心率,低压,高压,GCYY
0,2022-07-24 23:30:00,650.400000,92.316667,69.266667,112.400000,0.011171
1,2022-07-24 23:31:00,647.300000,91.883333,68.933333,111.683333,0.011224
2,2022-07-24 23:32:00,635.050847,93.949153,69.474576,113.711864,0.011047
3,2022-07-24 23:33:00,646.800000,92.900000,69.350000,113.083333,0.011118
4,2022-07-24 23:34:00,634.983333,93.083333,69.233333,112.066667,0.011149
...,...,...,...,...,...,...
1268,2022-07-25 22:51:00,650.283019,91.000000,71.528302,123.811321,0.010748
1269,2022-07-25 22:52:00,643.633333,92.766667,71.800000,124.716667,0.010640
1270,2022-07-25 22:53:00,496.769231,94.461538,72.538462,126.615385,0.010486
1271,2022-07-25 22:54:00,949.000000,95.000000,74.000000,130.000000,0.010305


In [162]:

# df = df_all.copy()
# # print(df["时间"])
# df['time'] = pd.to_datetime(df['时间'], format='%Y-%m-%d %H:%M:%S')
# display(df)
# df_plt = pd.concat([df['time'], pd.DataFrame({'GCYY': arr_GCYY},)], axis=1, join='inner')
# display(df_plt.index)
#Import Library
# import matplotlib.pyplot as plt
# import matplotlib.ticker as tck
# import matplotlib.dates as mdates
# Plot scatter
# plt.scatter(df_plt.index, df_plt["GCYY"], color = 'gray')
# fig = plt.figure(figsize=(20,5))
# ax = fig.add_subplot()  
# greenIndex = indexrGreen[0]
# # print(np.where(arr_GCYY))
# # print(df['time'].iloc[greenIndex])
# plt.scatter(df['time'].iloc[greenIndex], arr_GCYY[greenIndex], color = 'green')
# orangeIndex = indexrOrange[0]
# # print(arr_GCYY[orangeIndex])
# # print(df['time'].iloc[orangeIndex])
# plt.scatter(df['time'].iloc[orangeIndex], arr_GCYY[orangeIndex], color = 'orange')
# restIndices = np.setdiff1d(np.where(arr_GCYY), greenIndex)
# restIndices = np.setdiff1d(restIndices, orangeIndex)
# # print(restIndices)
# plt.scatter(df['time'].iloc[restIndices], arr_GCYY[restIndices], color = 'gray')

# ax.xaxis.set_major_locator(mdates.MinuteLocator(interval=20))

# # Labelling 
# plt.xlabel("Time")
# plt.ylabel("GCYY")
# # Auto space
# plt.tight_layout()
# fig.autofmt_xdate()
# # Display
# plt.show()


# df_plot=pd.DataFrame({'x': df['time'], 'y': arr_GCYY})
# datenums=md.date2num(df['time'])
# # plt.plot( 'x', 'y', "", data=df_plot, linestyle='', marker='o', markersize=0.9, color = 'orange')
# N = len(df['time'])
# def format_date(x, pos=None):
#     thisind = np.clip(int(x + 0.5), 0, N - 1)
#     return df['time'][thisind].item().strftime('%Y-%m-%d')
# # plt.xticks(np.arange(min(df['time']), max(df['time'])+timedelta(days=1), 1))
# plt.subplots_adjust(bottom=0.2)
# plt.xticks( rotation=25 )
# ax=plt.gca()
# xfmt = md.DateFormatter('%Y-%m-%d %H:%M:%S')
# ax.xaxis.set_major_formatter(xfmt)
# plt.xlabel('Value of X')
# plt.ylabel('Value of Y')
# plt.plot(datenums,arr_GCYY,linestyle='',marker='o', markersize=0.9, color = 'orange')
# plt.show()
